<a href="https://colab.research.google.com/github/anabiancachiricu/BigDataProject/blob/main/BigDataProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proiect Big data - Chiricu Ana Bianca, grupa 405

1. Introducere

Am ales setul de date Netflix Movies and TV Shows care reprezinta o lista cu toate filmele si serialele disponibile pe platforma de streaming Netflix la mmomentul crearii setului de date, impreuna cu detalii despre fiecare, cum ar fi durata, anul lansarii, distributia sau regizorii.



Link data set: https://www.kaggle.com/datasets/shivamb/netflix-shows

2.

In [5]:
!pip install PySpark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2519246f42895c3fbfdda9f78020a39ad9acac372da2bf518098ac4dd6a6b286
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built PySpark


In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import seaborn as sns
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()

In [8]:
dataframe = spark.read.csv('sample_data/netflix_titles.csv', inferSchema=True, header=True)
dataframe.show(10)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [9]:
dataframe.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [10]:
dataframe.registerTempTable('NetflixTable')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Verificarea si eliminarea inregistrarilor care au alt tip in afara de tv show si movie.

In [11]:
df2= spark.sql("Select distinct (type) from NetflixTable" )
df2.show()

+-------------+
|         type|
+-------------+
|         null|
|      TV Show|
|        Movie|
|William Wyler|
+-------------+



In [12]:
df3= spark.sql("Select * from NetflixTable where type in ('TV Show', 'Movie') ")

In [13]:
df3.show(10)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [14]:
df3.registerTempTable('NetflixTable')

Eliminarea inregistrarilor care nu au an de lansare corespunzator.

In [15]:
df4=spark.sql("Select distinct(release_year) from NetflixTable")
df4.show()

+-----------------+
|     release_year|
+-----------------+
|     Ted Ferguson|
|             1987|
|             1956|
|             2016|
|             2020|
|             2012|
|             1958|
|             1943|
|             1972|
| Marquell Manning|
|             1988|
|             2019|
|             2017|
|             1977|
|             2014|
|             1971|
|             1984|
|             2013|
|             1982|
|             2005|
+-----------------+
only showing top 20 rows



In [16]:
df4= spark.sql("Select * from NetflixTable where release_year like ('____') order by release_year ")
df4.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|       date_added|release_year|rating|duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|  s4251|TV Show|Pioneers: First W...|                null|                null|                null|December 30, 2018|        1925| TV-14|1 Season|            TV Shows|This collection r...|
|  s7791|  Movie|      Prelude to War|         Frank Capra|                null|       United States|   March 31, 2017|        1942| TV-14|  52 min|Classic Movies, D...|Frank Capra's doc...|
|  s8206|  Movie|The Battle of Midway|       

In [17]:
df4.registerTempTable('NetflixTable')

Eliminarea tuturor inregistrarilor care contin date lipsa.

In [18]:
dfclean= spark.sql("Select * from NetflixTable where director is not null and cast is not null and country is not null and date_added is not null and release_year is not null and rating is not null and duration is not null and listed_in is not null and description is not null ")
dfclean.show()

+-------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|            director|                cast|             country|       date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------+--------+--------------------+--------------------+
|  s8206|Movie|The Battle of Midway|           John Ford|Henry Fonda, Jane...|       United States|   March 31, 2017|        1942| TV-14|  18 min|Classic Movies, D...|Director John For...|
|  s8641|Movie|    Tunisian Victory|Frank Capra, John...|    Burgess Meredith|United States, Un...|   March 31, 2017|        1944| TV-14|  76 min|Classic Movies, D...|British and Ameri...|
|  s7220|Movie|Know Your Enemy -...|Frank Capra, Jori..

In [19]:
dfclean.registerTempTable('NetflixTableClean')

Calcularea numarului de filme si seriale aparute in fiecare an

In [20]:
dfmoviesandshowsperyear= spark.sql("select release_year, count(show_id),  type from NetflixTableClean group by release_year, type order by release_year" )
dfmoviesandshowsperyear.show()

+------------+--------------+-----+
|release_year|count(show_id)| type|
+------------+--------------+-----+
|        1942|             1|Movie|
|        1944|             1|Movie|
|        1945|             1|Movie|
|        1946|             1|Movie|
|        1947|             1|Movie|
|        1954|             2|Movie|
|        1955|             3|Movie|
|        1956|             2|Movie|
|        1958|             3|Movie|
|        1959|             1|Movie|
|        1960|             3|Movie|
|        1961|             1|Movie|
|        1962|             3|Movie|
|        1963|             1|Movie|
|        1964|             2|Movie|
|        1965|             2|Movie|
|        1966|             1|Movie|
|        1967|             4|Movie|
|        1968|             3|Movie|
|        1969|             2|Movie|
+------------+--------------+-----+
only showing top 20 rows



Durata medie a filmelor in functie de anul lansarii

In [21]:
from pyspark.sql.functions import first
from pyspark.sql.functions import split

df7 = spark.sql("Select type, release_year, duration from NetflixTableClean")
dff = df7.groupBy("type", "release_year", "duration").agg(split("duration", " ").getItem(0).alias("dur"))
dff.registerTempTable('NetflixMoviesDuration')
dfavgDuration=spark.sql("Select avg(dur), release_year from NetflixMoviesDuration group by release_year order by release_year")
dfavgDuration.show()

+------------------+------------+
|          avg(dur)|release_year|
+------------------+------------+
|              18.0|        1942|
|              76.0|        1944|
|              63.0|        1945|
|              58.0|        1946|
|              42.0|        1947|
|             118.0|        1954|
|100.33333333333333|        1955|
|             101.0|        1956|
| 98.66666666666667|        1958|
|             143.0|        1959|
|             131.0|        1960|
|             156.0|        1961|
|             167.0|        1962|
|             194.0|        1963|
|             200.5|        1964|
|             151.5|        1965|
|             116.0|        1966|
|            105.25|        1967|
|149.66666666666666|        1968|
|             140.0|        1969|
+------------------+------------+
only showing top 20 rows



3. Regresie logistica pentru predictia ratingului in functie de tara, catgorie si anul aparitiei.

In [22]:
import pyspark.ml
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression

Prelucrarea datelor:

In [23]:
df= spark.sql("Select title, director, cast, listed_in, rating, country, cast(release_year as int) from NetflixTableClean")

In [24]:
display(df)

DataFrame[title: string, director: string, cast: string, listed_in: string, rating: string, country: string, release_year: int]

Verificam cate filme si seriale fac parte din fiecare categorie de rating

In [25]:
df.groupby("rating").count().show()

+--------+-----+
|  rating|count|
+--------+-----+
|    TV-Y|   76|
|      UR|    3|
|      PG|  274|
|   TV-MA| 1812|
|TV-Y7-FV|    3|
|      NR|   58|
|   TV-PG|  431|
|   NC-17|    2|
|       R|  775|
|       G|   40|
|   TV-14| 1213|
|    TV-G|   84|
|   TV-Y7|   76|
|   PG-13|  469|
+--------+-----+



Functie definita de utilizator pentru a imparti categoriile de rating in 3 categorii: children, teenagers si adults.

In [26]:
from pyspark.sql.functions import udf
def udf_content_rating_group(rating):
  if rating == "TV-Y" or rating == "TV-Y7":
    return 'Children'
  elif rating == "TV-MA" or rating == "NC-17" or rating == "R" or rating == "TV-G":
    return 'Adults'
  elif rating =="TV-Y7-FV" or rating == "TV-PG" or rating== "TV-14"  or rating == "PG" or rating == "G" or rating == "PG-13":
    return 'Teenagers'
  else:
    return 'N/A'

rating_udf= udf(udf_content_rating_group)
df=df.withColumn("rating_udf",rating_udf('rating') )
df.show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+
|               title|            director|                cast|           listed_in|rating|             country|release_year|rating_udf|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+
|The Battle of Midway|           John Ford|Henry Fonda, Jane...|Classic Movies, D...| TV-14|       United States|        1942| Teenagers|
|    Tunisian Victory|Frank Capra, John...|    Burgess Meredith|Classic Movies, D...| TV-14|United States, Un...|        1944| Teenagers|
|Know Your Enemy -...|Frank Capra, Jori...|Walter Huston, Da...|Classic Movies, D...| TV-14|       United States|        1945| Teenagers|
|  Let There Be Light|         John Huston|       Walter Huston|Classic Movies, D...| TV-PG|       United States|        1946| Teenagers|
|         Thunderbolt|William Wyle

In [27]:
stringIndexer=StringIndexer().setInputCol("rating_udf").setOutputCol("rating_udfIndex")
rating_udfIndex_model=stringIndexer.fit(df)
rating_udfIndex_df = rating_udfIndex_model.transform(df)
rating_udfIndex_df.groupby("rating_udfIndex").count().show()

+---------------+-----+
|rating_udfIndex|count|
+---------------+-----+
|            0.0| 2673|
|            1.0| 2430|
|            3.0|   61|
|            2.0|  152|
+---------------+-----+



Utilizerea de StringIndexer pentru restul coloanelor: country, director, listed_in

In [28]:
stringIndexer=StringIndexer().setInputCol("country").setOutputCol("country_indexed")
country_indexed_model=stringIndexer.fit(rating_udfIndex_df)
country_indexed_df = country_indexed_model.transform(rating_udfIndex_df)
country_indexed_df.groupby("country_indexed").count().show()

+---------------+-----+
|country_indexed|count|
+---------------+-----+
|          305.0|    1|
|          558.0|    1|
|          596.0|    1|
|          496.0|    1|
|          299.0|    1|
|          576.0|    1|
|          147.0|    2|
|          184.0|    1|
|          170.0|    1|
|          160.0|    1|
|          169.0|    1|
|           70.0|    4|
|           67.0|    4|
|          571.0|    1|
|            8.0|   79|
|          486.0|    1|
|          379.0|    1|
|          311.0|    1|
|          168.0|    1|
|            0.0| 1836|
+---------------+-----+
only showing top 20 rows



In [29]:
stringIndexer=StringIndexer().setInputCol("director").setOutputCol("director_indexed")
director_indexed_model=stringIndexer.fit(country_indexed_df)
director_indexed_df = director_indexed_model.transform(country_indexed_df)
director_indexed_df.groupby("director_indexed").count().show()

+----------------+-----+
|director_indexed|count|
+----------------+-----+
|           934.0|    1|
|          2734.0|    1|
|           596.0|    2|
|          1761.0|    1|
|           299.0|    2|
|           558.0|    2|
|          2862.0|    1|
|           769.0|    1|
|          2815.0|    1|
|           305.0|    2|
|          1051.0|    1|
|          3597.0|    1|
|          3901.0|    1|
|           692.0|    2|
|           496.0|    2|
|          3029.0|    1|
|           720.0|    2|
|           782.0|    1|
|           576.0|    2|
|          3507.0|    1|
+----------------+-----+
only showing top 20 rows



In [30]:
stringIndexer=StringIndexer().setInputCol("listed_in").setOutputCol("listed_in_indexed")
listed_in_indexed_model=stringIndexer.fit(director_indexed_df)
listed_in_indexed_df = listed_in_indexed_model.transform(director_indexed_df)
listed_in_indexed_df.groupby("listed_in_indexed").count().show()

+-----------------+-----+
|listed_in_indexed|count|
+-----------------+-----+
|            299.0|    1|
|            305.0|    1|
|            147.0|    4|
|            184.0|    2|
|            170.0|    3|
|            160.0|    3|
|            169.0|    3|
|             67.0|   11|
|              8.0|  143|
|             70.0|   11|
|            311.0|    1|
|            168.0|    3|
|              0.0|  335|
|             69.0|   11|
|            206.0|    2|
|              7.0|  152|
|            249.0|    1|
|            142.0|    4|
|            191.0|    2|
|            329.0|    1|
+-----------------+-----+
only showing top 20 rows



In [31]:
listed_in_indexed_df.show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+
|               title|            director|                cast|           listed_in|rating|             country|release_year|rating_udf|rating_udfIndex|country_indexed|director_indexed|listed_in_indexed|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+
|The Battle of Midway|           John Ford|Henry Fonda, Jane...|Classic Movies, D...| TV-14|       United States|        1942| Teenagers|            1.0|            0.0|          2151.0|            120.0|
|    Tunisian Victory|Frank Capra, John...|    Burgess Meredith|Classic Movies, D...| TV-14|United States, Un...|        1944| Teenagers|            1.0|           22.0|          1

Adaugarea oneHotEncoder pentru grupele de rating si pentru titlu, regizor si tara

In [32]:
encoder=OneHotEncoder().setInputCols(["rating_udfIndex","country_indexed", "director_indexed", "listed_in_indexed" ]).setOutputCols(["rating_encoded", "country_encoded", "director_encoded", "listed_in_encoded"])
encoder_model=encoder.fit(listed_in_indexed_df)
encoder_df=encoder_model.transform(listed_in_indexed_df)

encoder_df.show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+
|               title|            director|                cast|           listed_in|rating|             country|release_year|rating_udf|rating_udfIndex|country_indexed|director_indexed|listed_in_indexed|rating_encoded|  country_encoded|   director_encoded|listed_in_encoded|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+
|The Battle of Midway|           John Ford|Henry Fonda, Jane...|Classic Movies, D...| TV-14|       United States|        1942| Teenagers|            1.0|            0.0|   

Transformarea caracteristiciolr selectate in vector folosind Vector Assembler

In [67]:
assembler=VectorAssembler().setInputCols([ "country_encoded", "director_encoded", "listed_in_encoded",  "release_year"]).setOutputCol("vectorized_features_out")
assembler_df=assembler.transform(encoder_df)
assembler_df.show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+-----------------------+
|               title|            director|                cast|           listed_in|rating|             country|release_year|rating_udf|rating_udfIndex|country_indexed|director_indexed|listed_in_indexed|rating_encoded|  country_encoded|   director_encoded|listed_in_encoded|vectorized_features_out|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+-----------------------+
|The Battle of Midway|           John Ford|Henry Fonda, Jane...|Classic Movies, D...| TV-14|       U

Adaugarea de labels

In [68]:
label_indexer = StringIndexer().setInputCol("rating_udfIndex").setOutputCol("label")
label_indexer_model = label_indexer.fit(assembler_df)
label_indexer_df= label_indexer_model.transform(assembler_df)

label_indexer_df.show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+-----------------------+-----+
|               title|            director|                cast|           listed_in|rating|             country|release_year|rating_udf|rating_udfIndex|country_indexed|director_indexed|listed_in_indexed|rating_encoded|  country_encoded|   director_encoded|listed_in_encoded|vectorized_features_out|label|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+-----------------------+-----+
|The Battle of Midway|           John Ford|Henry Fonda, Jane...|Classic Movies, D.

In [34]:
from pyspark.ml.feature import StandardScaler

In [69]:
scaler=StandardScaler()
scaler.setInputCol("vectorized_features_out")
scaler.setOutputCol("features")
scaler_model=scaler.fit(label_indexer_df)
scaler_df=scaler_model.transform(label_indexer_df)
scaler_df.show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+-----------------------+-----+--------------------+
|               title|            director|                cast|           listed_in|rating|             country|release_year|rating_udf|rating_udfIndex|country_indexed|director_indexed|listed_in_indexed|rating_encoded|  country_encoded|   director_encoded|listed_in_encoded|vectorized_features_out|label|            features|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+------------+----------+---------------+---------------+----------------+-----------------+--------------+-----------------+-------------------+-----------------+-----------------------+-----+--------------------+
|The Battle of Midw

Imparrtirea setului de date in test si train

In [70]:
train, test= scaler_df.randomSplit([0.8, 0.2], seed = 2018)
print("Training:" + str(train.count()))
print("Testing:" + str(test.count()))


Training:4292
Testing:1024


In [71]:
train.groupby("rating_udf").count().show()

+----------+-----+
|rating_udf|count|
+----------+-----+
| Teenagers| 1959|
|       N/A|   51|
|  Children|  126|
|    Adults| 2156|
+----------+-----+



Antrenarea modelului:

In [53]:
from pyspark.ml.classification import LogisticRegression

In [72]:
lr=LogisticRegression(featuresCol='features', labelCol='label', maxIter=5)
lrModel=lr.fit(train)
predictions = lrModel.transform(test)
predictions.select('label', 'features', 'rawPrediction', 'prediction', 'probability' ).toPandas().head(5)

,label,features,rawPrediction,prediction,probability
0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.7966739920552257, 1.0362036439696425, -1.71...",0.0,"[0.8401296497697716, 0.14447202306579301, 0.00..."
1,1.0,"(2.1028977685248162, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.6196192391916213, 1.9423363031701513, -1.47...",1.0,"[0.40804284660263834, 0.5634560011426484, 0.01..."
2,1.0,"(2.1028977685248162, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5513024980409824, 2.9429496054799955, -1.35...",1.0,"[0.08231841204405566, 0.8998624413881202, 0.01..."
3,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.073586194094901, -0.2376074450211263, -1.52...",0.0,"[0.9815349899732081, 0.01316976962840571, 0.00..."
4,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.7198178217476112, 2.136823399450833, -1.563...",1.0,"[0.38865100546530834, 0.5897433017517467, 0.01..."


Calcularea acuratetii


In [73]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count()/float(predictions.count())
print("Accuracy:", accuracy)

Accuracy: 0.6767578125


4. Adaugarea unui pipeline

In [37]:
from pyspark.ml import Pipeline
